# Работу выполнил Бедак Иван ПИ19-4
# Вариант 77

In [138]:
import numpy as np
#Матрица переходов
P = np.array([
    [0, 0.2, 0, 0, 0.73, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #1
    [0.16, 0, 0, 0.04, 0.08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #2
    [0, 0.27, 0, 0.16, 0, 0.22, 0.12, 0, 0, 0, 0, 0, 0, 0, 0, 0], #3
    [0.18, 0.39, 0, 0, 0, 0, 0, 0, 0, 0.05, 0.33, 0, 0, 0, 0, 0], #4
    [0.19, 0, 0, 0.57, 0, 0.17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #5
    [0.26, 0.02, 0, 0, 0.22, 0, 0.13, 0.13, 0.1, 0.13, 0, 0, 0, 0, 0, 0], #6
    [0, 0, 0.37, 0, 0, 0.33, 0, 0, 0.23, 0, 0, 0, 0, 0, 0, 0], #7
    [0, 0, 0, 0.15, 0, 0, 0, 0, 0.23, 0, 0.22, 0, 0, 0.45, 0, 0], #8
    [0, 0, 0, 0.31, 0.27, 0, 0, 0.37, 0, 0, 0, 0, 0, 0, 0, 0], #9
    [0, 0, 0, 0.29, 0.2, 0, 0.2, 0, 0, 0, 0.08, 0.19, 0, 0, 0, 0], #10
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.47], #11
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.83, 0, 0, 0, 0, 0.08, 0], #12
    [0, 0, 0, 0, 0, 0, 0, 0, 0.24, 0, 0, 0.35, 0, 0, 0, 0.37], #13
    [0, 0, 0, 0, 0, 0, 0, 0.27, 0.18, 0.13, 0, 0, 0, 0, 0.25, 0], #14
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.37, 0.16, 0, 0, 0], #15
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.15, 0, 0, 0.77, 0] #16
    ])

#Добавление вероятностей задержаться в вершине
for i in range(len(P)):
    P[i][i] = 1 - sum(P[i])

#P


In [139]:
#1) вероятность того, что за 6 шагов система перейдет из состояния 12 в состояние 4
k = 6
i = 12
j = 4
result = P.copy()
for _ in range(k-1):
    result = result @ P

print(result[i - 1][j - 1])

0.10255186107299998


In [140]:
#2) вероятности состояний системы спустя 8 шагов, если в начальный момент вероятность состояний были следующими
A = np.array([[0.05,0.08,0.11,0.02,0.09,0.02,0.12,0.07,0.06,0.1,0.03,0.07,0,0.09,0.04,0.05]])
k = 8
for i in range(k):
    A = A @ P

print(A)

[[0.08839806 0.21780461 0.0087526  0.1048054  0.11290123 0.02739819
  0.01852223 0.01116745 0.01560609 0.06556128 0.09227696 0.06602665
  0.01558571 0.00681792 0.09397725 0.05439837]]


In [141]:
#3) вероятность первого перехода за 7 шагов из состояния 5 в состояние 9

def task3(p, k):
    result = np.zeros((k, len(p), len(p)))
    for n in range(k):
        for i in range(len(p)):
            for j in range(len(p)):
                if n == 0:
                    result[n][i][j] = p[i][j]
                for m in range(len(p)):
                    if m != j:
                        result[n][i][j] += p[i][m] * result[n - 1][m][j]
    return result
k = 7
i = 5
j = 9
result_3 = task3(P, k)
print(result_3[k - 1][i - 1][j - 1])

0.009682042793999998


In [142]:
%%time
result_10k = task3(P, 10000)

Wall time: 1min 5s


In [143]:
#4) вероятность перехода из состояния 8 в состояние 6 не позднее чем за 7 шагов


k = 7
i = 8
j = 7
temp = P.copy()
for _ in range(1, k):
    temp += result_10k[_]

print(temp[i - 1][j - 1])

0.07317425108562


In [144]:
#5) среднее количество шагов для перехода из состояния 5 в состояние 13

i = 5
j = 13

temp = P.copy()
for k in range(1, 10000):
    temp += k * result_10k[k]
print(temp[i - 1][j - 1])

77.35739578130307


In [145]:
#6) вероятность первого возвращения в состояние 8 за 7 шагов

def task6(p, k):
    if k == 1:
        return p
    else:
        temp = np.zeros((k, len(p), len(p)))
        for i in range(k):
            temp[i] = np.linalg.matrix_power(p, i + 1)
        fjj = temp.copy()
        for n in range(1, k):
            for m in range(n):
                fjj[n] = fjj[n] - fjj[m] * temp[n - m - 1]
        return fjj

k = 7
i = 8
    
return_10 = task6(P, 10)
print(return_10[k - 1][i - 1][i - 1])

0.006443334313029999


In [146]:
#7) вероятность возвращения в состояние 6 не позднее чем за 8 шагов

k = 8
i = 6

result = 0
for j in range(1, k + 1):
    result += return_10[j - 1][i - 1][i - 1]

print(result)

0.2578813720442695


In [147]:
%%time
return_10k = task6(P, 2000)

Wall time: 8.38 s


In [157]:
#8) среднее время возвращения в состояние 6
i = 6
result = 0
for j in range(1, 2001):
    result += j * return_10k[j - 1][i - 1][i - 1]
    #print(result)

print(result)

35.3349687368479


In [158]:
#9) установившиеся вероятности
P_ = np.copy(P).transpose()
for i in range(len(P_)):
    P_[i][i] -= 1
P_[-1, :] = 1

B = np.zeros(len(P_))
B[-1] = 1
X = np.linalg.inv(P_).dot(B)

print(X)

[0.0908216  0.22448386 0.00921813 0.10675962 0.115641   0.02830058
 0.01918367 0.01014672 0.01483689 0.06527783 0.09081953 0.06377651
 0.01470323 0.00550123 0.08821939 0.05231019]
